In [52]:
# Import modules/packages
import os
import numpy as np
from shutil import copyfile, copy

# Import third-party modules/packages
# import utils
from utils import File, Command, NiiFile, LogFile, TmpDir, DependencyError
from typing import List, Optional, Union, Tuple

In [2]:
# See these links for more info on improving tractograms:
# https://community.mrtrix.org/t/pipeline-for-multi-shell-data/1384/3

In [ ]:
# Compute connectome:
# https://mrtrix.readthedocs.io/en/latest/reference/commands/tck2connectome.html#tck2connectome

In [ ]:
# Compute FA and other measures:
# https://mrtrix.readthedocs.io/en/latest/reference/commands/dwi2tensor.html
# https://mrtrix.readthedocs.io/en/latest/reference/commands/tensor2metric.html

In [2]:
# scripts_dir = os.path.realpath(__file__)
scripts_dir = os.getcwd()

In [49]:
class MRtrixError(Exception):
    pass

In [50]:
class FSLError(Exception):
    pass

In [81]:
class ReconMRtrix (object):
    '''Class that contains the associated wrapper functions for performing
    tractography using MRtrix v3.x.
    
    Attributes:
        *
    
    To-do:
        * Write general purpose mif to nifti and (nifti to mif) conversion function(s)
    '''
    
    nii_file: NiiFile = ""
    bval: File = ""
    bvec: File = ""
    json_file: File = ""
    log: LogFile = ""
    
    def __init__(self,
                 nii_file: str,
                 bval: str,
                 bvec: str,
                 json_file: Optional[str] = None,
                 log: Optional[str] = None
                ) -> None:
        '''Init doc string for ReconMRtrix class.
        
        Usage example:
            >>> dwi_obj = ReconMRtrix("file.nii",
            >>>                       "file.bval",
            >>>                       "file.bvec",
            >>>                       "file.json",
            >>>                       "file.log")
            >>> dwi_obj.file
            "file.nii"
        
        Args:
            nii_file: Input file path to DWI NIFTI file.
            bval: Input file path to corresponding bval file.
            bvec: Input file path to corresponding bvec file.
            json_file: Input file path to corresponding JSON (sidecar) file.
            log: Log filename for output log file (, need not exist at runtime).
        '''
        self.nii_file: str = nii_file
        self.bval: str = bval
        self.bvec: str = bvec
        
        self.nii_file = NiiFile(self.nii_file)
        self.bval = File(self.bval)
        self.bvec = File(self.bvec)
        
        if json_file:
            self.json_file: str = json_file
            self.json_file = File(self.json_file)
        else:
            self.json_file = ""
            self.json_file = File(self.json_file)
            
        if log:
            self.log: str = log
            self.log = LogFile(self.log)
        else:
            self.log: str = ""
            self.log = LogFile(self.log)
        
    class Mif(File):
        '''Creates MIF files for use with MRtrix executables. Inherits 
        methods and properties from the File class.
        
        Attributes:
            * 
        '''
        
        def __init__(self,
                     file: File,
                     gzip: bool = False
                    ) -> None:
            '''Init doc string for Mif class. Inherits methods and properties
            from the File class. The MIF file can also be gzipped if desired.
            
            Usage example:
                >>> dwi_obj = ReconMRtrix("file.nii",
                >>>                       "file.bval",
                >>>                       "file.bvec",
                >>>                       "file.json",
                >>>                       "file.log")
                >>> dwi_obj.Mif(dwi_obj.file)
                >>> dwi_obj.Mif.file
                >>> "file.mif"
                >>>
                >>> # To gzip the MIF file
                >>> dwi_obj.Mif(dwi_obj.file,gzip=True)
            
            Args:
                file: Input filename of MIF file (need not exist).
                gzip: Gzips output MIF file
            '''
            self.file = file
            
            if '.gz' in self.file:
                if gzip:
                    self.ext: str = self.file[-(7):]
                    self.file = self.file[:-(7)] + ".mif.gz"
                else:
                    self.ext: str = self.file[-(7):]
                    self.file = self.file[:-(7)] + ".mif"
            else:
                if gzip:
                    self.ext: str = self.file[-(4):]
                    self.file = self.file[:-(4)] + ".mif.gz"
                else:
                    self.ext: str = self.file[-(4):]
                    self.file = self.file[:-(4)] + ".mif"
            File.__init__(self,self.file,self.ext)
    
    def nifti_to_mif(self,
                     force: bool = False,
                     gzip: bool = False
                    ) -> Mif:
        '''Converts DWI NIFTI file and its associated files to MIF files.
        
        Usage example:
            >>> dwi_obj = ReconMRtrix("file.nii",
            >>>                       "file.bval",
            >>>                       "file.bvec",
            >>>                       "file.json",
            >>>                       "file.log")
            >>> dwi_obj.nifti_to_mif()
            >>> Mif
        
        Args:
            force: Force overwrite of existing MIF file.
            gzip: Gzip output MIF file.
        
        Returns:
            mif_file: Mif file object.
        '''
        mif_file: str = self.nii_file.abs_path()
        mif_file: Mif = self.Mif(mif_file,gzip=gzip)
        
        mr_convert = Command("mrconvert")
        if force:
            mr_convert.cmd_list.append("-force")
        if self.json_file:
            mr_convert.cmd_list.append("-json_import")
            mr_convert.cmd_list.append(self.json_file.file)
        mr_convert.cmd_list.append("-fslgrad")
        mr_convert.cmd_list.append(f"{self.bvec.file}")
        mr_convert.cmd_list.append(f"{self.bval.file}")
        mr_convert.cmd_list.append(f"{self.nii_file.file}")
        mr_convert.cmd_list.append(f"{mif_file.file}")
        
        mr_convert.run(self.log)
        return mif_file
    
    def estimate_response(self,
                          mif: Mif,
                          method: str = "dhollander",
                          force: bool = False,
                          gzip: bool = False
                         ) -> Tuple[File,File,File]:
        '''doc-string
        Todo:
            * function returns should be File objects, not strs
        '''
        [path, filename, ext] = self.nii_file.file_parts()
        
        wm_res: str = os.path.join(path,filename + "_response_wm.txt")
        gm_res: str = os.path.join(path,filename + "_response_gm.txt")
        csf_res: str = os.path.join(path,filename + "_response_csf.txt")
            
        wm_res: File = File(wm_res)
        gm_res: File = File(gm_res)
        csf_res: File = File(csf_res)
        
        dwi_response = Command("dwi2response")
        if force:
            dwi_response.cmd_list.append("-force")
        dwi_response.cmd_list.append(method)
        dwi_response.cmd_list.append(mif.file)
        dwi_response.cmd_list.append(wm_res.file)
        dwi_response.cmd_list.append(gm_res.file)
        dwi_response.cmd_list.append(csf_res.file)
        
        dwi_response.run(self.log)
        
        return wm_res, gm_res, csf_res
    
    def mr_upsample(self,
                    mif: Mif,
                    vox: float,
                    gzip: bool = False
                   ) -> Mif:
        '''doc-string'''
        [path, filename, _ext] = self.nii_file.file_parts()
        [_path, _filename, ext] = mif.file_parts() # Keep original file extension
        
        if ".gz" in mif.file:
            gzip = True
        
        filename = filename + "_upsampled"
        upsampled_mif = os.path.join(path,filename + ext)
        upsampled_mif: Mif = self.Mif(upsampled_mif,gzip=gzip)
        
        try:
            upsample = Command("mrresize")
            upsample.cmd_list.append(mif.file)
            upsample.cmd_list.append("-vox")
            upsample.cmd_list.append(f"{vox}")
            upsample.cmd_list.append(upsampled_mif.file)

            upsample.check_dependency()
            upsample.run(self.log)
            return upsampled_mif
        except DependencyError:
            upsample = Command("mrgrid") 
            upsample.cmd_list.append(mif.file)
            upsample.cmd_list.append("regrid")
            upsample.cmd_list.append(upsampled_mif.file)
            upsample.cmd_list.append("-voxel")
            upsample.cmd_list.append(f"{vox}")
            
            upsample.check_dependency()
            upsample.run(self.log)
            return upsampled_mif
    
    def create_mask(self,
                    mif: Mif,
                    frac_int: float = 0.5,
                    gzip: bool = False,
                    cleanup: bool = True
                   ) -> Tuple[Mif,Mif,Mif]:
        '''doc-string'''
        [path, _filename, _ext] = self.nii_file.file_parts()
        [_path, filename, ext] = mif.file_parts() # Keep original file extension
        
        if ".gz" in mif.file:
            gzip = True
        
        file_head = filename + "_head"
        filename = filename + "_brain"
        file_mask = filename + "_mask"
        
        brain_mif = os.path.join(path,filename + ext)
        mask_mif = os.path.join(path,file_mask + ext)
        head_mif = os.path.join(path,file_head + ext)
        
        brain_mif = self.Mif(brain_mif,gzip=gzip)
        mask_mif: Mif = self.Mif(mask_mif,gzip=gzip)
        head_mif: Mif = self.Mif(head_mif,gzip=gzip)
            
        # Create temporary directory
        work_dir: TmpDir = TmpDir(tmp_dir=path,use_cwd=False)
        work_dir.mk_tmp_dir()
        
        tmp_b0s: TmpFile = work_dir.TmpFile(tmp_file="tmp_B0s" + _ext,
                                            tmp_dir=work_dir.tmp_dir)
        
        tmp_b0: TmpFile = work_dir.TmpFile(tmp_file="tmp_B0" + _ext,
                                           tmp_dir=work_dir.tmp_dir)
        
        tmp_mask: TmpFile = work_dir.TmpFile(tmp_file=tmp_b0.rm_ext(ext=_ext) + 
                                             "_brain" + _ext,
                                             tmp_dir=work_dir.tmp_dir)
        
        # Extract B0s
        extract_b0s = Command("dwiextract")
        extract_b0s.cmd_list.append("-bzero")
        extract_b0s.cmd_list.append(mif.file)
        extract_b0s.cmd_list.append(tmp_b0s.file)
        extract_b0s.run(self.log)
        
        # Merge B0s, by obtaining mean of images
        merge_b0s = Command("fslmaths")
        merge_b0s.cmd_list.append(tmp_b0s.file)
        merge_b0s.cmd_list.append("-Tmean")
        merge_b0s.cmd_list.append(tmp_b0.file)
        merge_b0s.run(self.log)
        
        # Create brain mask
        bet = Command("bet")
        bet.cmd_list.append(tmp_b0.file)
        bet.cmd_list.append(tmp_mask.file)
        bet.cmd_list.append("-R")
        bet.cmd_list.append("-m")
        bet.cmd_list.append("-f")
        bet.cmd_list.append(f"{frac_int}")
        bet.run(self.log)
        
        # Convert NIFTI to MIF
        mr_convert = Command("mrconvert")
        mr_convert.cmd_list.append(tmp_mask.rm_ext(ext=_ext) + "_mask.nii.gz")
        mr_convert.cmd_list.append(mask_mif.file)
        mr_convert.run(self.log)
        
        mr_convert = Command("mrconvert")
        mr_convert.cmd_list.append(tmp_mask.file)
        mr_convert.cmd_list.append(brain_mif.file)
        mr_convert.run(self.log)
        
        mr_convert = Command("mrconvert")
        mr_convert.cmd_list.append(tmp_b0.file)
        mr_convert.cmd_list.append(head_mif.file)
        mr_convert.run(self.log)
        
        # Clean-up
        if cleanup:
            work_dir.rm_tmp_dir(rm_parent=False)
        
        return mask_mif,brain_mif,head_mif
        
    def ss3t_csd(self,
                 mif: Mif,
                 mask: Mif,
                 wm_res: File,
                 gm_res: File,
                 csf_res: File,
                 gzip: bool = False) -> Tuple[Mif,Mif,Mif]:
        '''doc-string'''
        [path, _filename, _ext] = self.nii_file.file_parts()
        [_path, filename, ext] = mif.file_parts() # Keep original file extension
        
        if ".gz" in mif.file:
            gzip = True
        
        wm_fod = filename + "_wm_fod"
        gm_tis = filename + "_gm_tis"
        csf_tis = filename + "_csf_tis"
        
        wm_fod: str = os.path.join(path,wm_fod + ext)
        gm_tis: str = os.path.join(path,gm_tis + ext)
        csf_tis: str = os.path.join(path,csf_tis + ext)
        
        wm_fod: Mif = self.Mif(wm_fod,gzip=gzip)
        gm_tis: Mif = self.Mif(gm_tis,gzip=gzip)
        csf_tis: Mif = self.Mif(csf_tis,gzip=gzip)
        
        # Compute WM FOD
        ss3t = Command("ss3t_csd_beta1")
        ss3t.cmd_list.append(mif.file)
        ss3t.cmd_list.append(wm_res.file)
        ss3t.cmd_list.append(wm_fod.file)
        ss3t.cmd_list.append(gm_res.file)
        ss3t.cmd_list.append(gm_tis.file)
        ss3t.cmd_list.append(csf_res.file)
        ss3t.cmd_list.append(csf_tis.file)
        ss3t.cmd_list.append("-mask")
        ss3t.cmd_list.append(mask.file)
        ss3t.run(self.log)
        return wm_fod, gm_tis, csf_tis
    
    def bias_field_correction(self,
                              wm_fod: Mif,
                              gm_tis: Mif,
                              csf_tis: Mif,
                              mask: Mif, 
                              gzip: bool = False
                             ) -> Tuple[Mif,Mif,Mif]:
        '''doc-string'''
        [path, filename, _ext] = self.nii_file.file_parts()
        
        if ".gz" in wm_fod.file:
            gzip = True
            ext = ".mif.gz"
        else:
            ext = ".mif"
        
        wm_fod_norm = filename + "_wm_fod_norm"
        gm_tis_norm = filename + "_gm_tis_norm"
        csf_tis_norm = filename + "_csf_tis_norm"
        
        wm_fod_norm: str = os.path.join(path,wm_fod_norm + ext)
        gm_tis_norm: str = os.path.join(path,gm_tis_norm + ext)
        csf_tis_norm: str = os.path.join(path,csf_tis_norm + ext)
        
        wm_fod_norm: Mif = self.Mif(wm_fod_norm,gzip=gzip)
        gm_tis_norm: Mif = self.Mif(gm_tis_norm,gzip=gzip)
        csf_tis_norm: Mif = self.Mif(csf_tis_norm,gzip=gzip)
            
        # Perform joint bias field correction
        bias_correct = Command("mtnormalise")
        bias_correct.cmd_list.append(wm_fod.file)
        bias_correct.cmd_list.append(wm_fod_norm.file)
        bias_correct.cmd_list.append(gm_tis.file)
        bias_correct.cmd_list.append(gm_tis_norm.file)
        bias_correct.cmd_list.append(csf_tis.file)
        bias_correct.cmd_list.append(csf_tis_norm.file)
        bias_correct.cmd_list.append("-mask")
        bias_correct.cmd_list.append(mask.file)
        bias_correct.run(self.log)
        return wm_fod_norm, gm_tis_norm, csf_tis_norm
    
    def compute_dec_map(self,
                        wm_fod: Mif,
                        gm_tis: Mif,
                        csf_tis: Mif,
                        mask: Mif,
                        gzip: bool = False) -> Tuple[Mif,Mif]:
        '''doc-string
        Todo:
            * write shell script to produce vf maps
        '''
        [path, filename, _ext] = self.nii_file.file_parts()
        
        if ".gz" in wm_fod.file:
            gzip = True
            ext = ".mif.gz"
        else:
            ext = ".mif"
        
        # Create output filenames
        dec = os.path.join(path,filename + "_dec" + ext)
        dec: Mif = self.Mif(dec,gzip=gzip)
        
        vf = os.path.join(path,filename + "_vf" + ext)
        vf: Mif = self.Mif(vf,gzip=gzip)
            
        # Compute DEC map
        fod2dec = Command("fod2dec")
        fod2dec.cmd_list.append(wm_fod.file)
        fod2dec.cmd_list.append(dec.file)
        fod2dec.cmd_list.append("-mask")
        fod2dec.cmd_list.append(mask.file)
        fod2dec.run(self.log)
        
        # Compute RGB tissue (signal contribution) maps
        vf_calc = os.path.join(scripts_dir,"dwi_extra.sh")
        rgb = Command(vf_calc)
        rgb.cmd_list.append("--wm-fod")
        rgb.cmd_list.append(wm_fod.file)
        rgb.cmd_list.append("--gm")
        rgb.cmd_list.append(gm_tis.file)
        rgb.cmd_list.append("--csf")
        rgb.cmd_list.append(csf_tis.file)
        rgb.cmd_list.append("--out-vf")
        rgb.cmd_list.append(vf.file)
        rgb.run(self.log)
        
        return dec, vf
    
    def mr_tck_global(self,
                      wm_fod: Mif,
                      mask: Mif,
                      stream_lines: int = 100000, 
                      cutoff: float = 0.07) -> File:
        '''doc-string
        Todo:
            * investigate ACT for neonates
        '''
        [path, filename, _ext] = self.nii_file.file_parts()
        
        # Create output filenames
        tck: str = os.path.join(path,filename + f".{stream_lines}.streamlines" + ".tck")
        tck: File = File(tck)
        
        # Construct tracts
        tckgen = Command("tckgen")
        tckgen.cmd_list.append(wm_fod.file)
        tckgen.cmd_list.append(tck.file)
        tckgen.cmd_list.append("-seed_image")
        tckgen.cmd_list.append(mask.file)
        tckgen.cmd_list.append("-select")
        tckgen.cmd_list.append(f"{stream_lines}")
        tckgen.cmd_list.append("-cutoff")
        tckgen.cmd_list.append(f"{cutoff}")
        tckgen.run(self.log)
        
        return tck
    
    def mr_tck_sift(self,
                   tck: File,
                   wm_fod: Mif,
                   term: Optional[int] = None,
                   mask: Mif = "") -> File:
        '''doc-string'''
        [path, filename, _ext] = self.nii_file.file_parts()
        
        # Create output filenames
        tck_filt: str = os.path.join(path,filename + f".{term}.streamlines.filtered" + ".tck")
        tck_filt: File = File(tck_filt)
        
        # Filter tracks
        filt = Command("tcksift")
        filt.cmd_list.append(tck.file)
        filt.cmd_list.append(wm_fod.file)
        filt.cmd_list.append(tck_filt.file)
        
        if mask:
            filt.cmd_list.append("-proc_mask")
            filt.cmd_list.append(mask.file)
        
        if term:
            filt.cmd_list.append("-term_number")
            filt.cmd_list.append(f"{term}")
        
        filt.run(self.log)
        
        return tck_filt
    
    def compute_diff_metrics(self,
                             dwi: Mif,
                             mask: Optional[Mif] = None,
                             fa: bool = True,
                             md: bool = True,
                             ad: bool = True,
                             rd: bool = True,
                             cleanup = True,
                             force: bool = False
                            ) -> Tuple[Mif,Mif,Mif,Mif]:
        '''doc-string'''
        [path, filename, _ext] = self.nii_file.file_parts()
        
        if ".gz" in dwi.file:
            ext = ".mif.gz"
        else:
            ext = ".mif"
        
        # Output file names
        tensor: str = os.path.join(path,filename + f".diff_tensor" + ext)
        md_mif: str = os.path.join(path,filename + f".md" + ext)
        fa_mif: str = os.path.join(path,filename + f".fa" + ext)
        ad_mif: str = os.path.join(path,filename + f".ad" + ext)
        rd_mif: str = os.path.join(path,filename + f".rd" + ext)
        
        tensor: Mif = self.Mif(tensor)
        md_mif: Mif = self.Mif(md_mif)
        fa_mif: Mif = self.Mif(fa_mif)
        ad_mif: Mif = self.Mif(ad_mif)
        rd_mif: Mif = self.Mif(rd_mif)
        
        # Compute tensors
        tens = Command("dwi2tensor")
        tens.cmd_list.append(dwi.file)
        tens.cmd_list.append(tensor.file)
        
        if mask:
            tens.cmd_list.append("-mask")
            tens.cmd_list.append(mask.file)
        
        if force:
            tens.cmd_list.append("-force")
        
        tens.run(self.log)
        
        # Compute diffusion tensor metrics
        metric = Command("tensor2metric")
        metric.cmd_list.append(tensor.file)
        
        if md:
            metric.cmd_list.append("-adc")
            metric.cmd_list.append(md_mif.file)
        
        if fa:
            metric.cmd_list.append("-fa")
            metric.cmd_list.append(fa_mif.file)
        
        if ad:
            metric.cmd_list.append("-ad")
            metric.cmd_list.append(ad_mif.file)
            
        if rd:
            metric.cmd_list.append("-rd")
            metric.cmd_list.append(rd_mif.file)
        
        metric.run(self.log)
        
        # Clean-up
        if cleanup:
            os.remove(tensor.file)
        
        return fa_mif,md_mif,ad_mif,rd_mif
    
    def structural_connectome(self,
                              tck: File,
                              labels: File,
                              dwi: Optional[Mif] = None,
                              mask: Optional[Mif] = None,
                              fa: bool = False,
                              md: bool = False,
                              ad: bool = False,
                              rd: bool = False,
                              cleanup:bool = True,
                              force: bool = False
                             ) -> Tuple[File,File,File,File,File]:
        '''doc-string'''
        [path, filename, _ext] = self.nii_file.file_parts()
        
        # Create output filenames
        connectome: str = os.path.join(path,filename + f".structural_connectome" + ".txt")
        fa_connectome: str = os.path.join(path,filename + f".structural_connectome.fa_weighted" + ".txt")
        md_connectome: str = os.path.join(path,filename + f".structural_connectome.md_weighted" + ".txt")
        ad_connectome: str = os.path.join(path,filename + f".structural_connectome.ad_weighted" + ".txt")
        rd_connectome: str = os.path.join(path,filename + f".structural_connectome.rd_weighted" + ".txt")
        tmp_metric: str = os.path.join(path,filename + f"metric.vertex.mean.csv")
        
        connectome: File = File(connectome)
        fa_connectome: File = File(fa_connectome)
        md_connectome: File = File(md_connectome)
        ad_connectome: File = File(ad_connectome)
        rd_connectome: File = File(rd_connectome)
        tmp_metric: File = File(tmp_metric)
            
        if fa or md or ad or rd:
            if dwi:
                [fa_mif, \
                 md_mif, \
                 ad_mif, \
                 rd_mif] = self.compute_diff_metrics(dwi=dwi,
                                                     mask=mask,
                                                     fa=fa,
                                                     md=md,
                                                     ad=ad,
                                                     rd=rd,
                                                     cleanup=cleanup,
                                                     force=force)
            else:
                MRtrixError("Input DWI mif file was not present.")
            
        # Compute structural connectome
        ctm = Command("tck2connectome")
        ctm.cmd_list.append(tck.file)
        ctm.cmd_list.append(labels.file)
        ctm.cmd_list.append(connectome.file)
        if force:
            ctm.cmd_list.append("-force")
        ctm.run(self.log)
        del ctm
        
        # Construct list of metrics
        metrics: List[Mif] = []
        out_files: List[File] = []
            
        if fa:
            metrics.append(fa_mif)
            out_files.append(fa_connectome)
        if md:
            metrics.append(md_mif)
            out_files.append(md_connectome)
        if ad:
            metrics.append(ad_mif)
            out_files.append(ad_connectome)
        if rd:
            metrics.append(rd_mif)
            out_files.append(rd_connectome)
        
        # Compute weighted structural connectome(s)
        if fa or md or ad or rd:
            for item in zip(metrics,out_files):
                # Sample mean vertex value
                tck_smp = Command("tcksample")
                tck_smp.cmd_list.append(tck.file)
                tck_smp.cmd_list.append(item[0].file)
                tck_smp.cmd_list.append(tmp_metric.file)
                tck_smp.cmd_list.append("-stat_tck")
                tck_smp.cmd_list.append("mean")
                # tck_smp.cmd_list.append("-force")
                tck_smp.run(self.log)
                del tck_smp

                # Compute weighted connectome
                ctm = Command("tck2connectome")
                ctm.cmd_list.append(tck.file)
                ctm.cmd_list.append(labels.file)
                ctm.cmd_list.append(item[1].file)
                ctm.cmd_list.append("-scale_file")
                ctm.cmd_list.append(tmp_metric.file)
                ctm.cmd_list.append("-stat_edge")
                ctm.cmd_list.append("mean")
                ctm.run(self.log)
                del ctm
                
                os.remove(tmp_metric.file)
        
        return connectome,fa_connectome,md_connectome,ad_connectome,rd_connectome
    
    # write visualization functions later

In [4]:
class DWIxfm(object):
    '''class doc-string'''
    
    dwi_file: NiiFile = ""
    dwi_bval: File = ""
    dwi_bvec: File = ""
    dwi_json: File = ""
    
    template: NiiFile = ""
    template_brain:NiiFile = ""
    labels: NiiFile = ""
    
    log: LogFile = ""
    
    def __init__(self,
                 dwi_file: str,
                 dwi_bval: str,
                 dwi_bvec: str,
                 template: str,
                 template_brain: str,
                 labels: str,
                 log:str,
                 dwi_json: str = ""):
        '''doc-string'''
        self.dwi_file: str = dwi_file
        self.dwi_bval: str = dwi_bval
        self.dwi_bvec: str = dwi_bvec
        self.dwi_json: str = dwi_json
        self.template: str = template
        self.template_brain: str = template_brain
        self.labels: str = labels
        self.log: str = log

        self.dwi_file: NiiFile = NiiFile(self.dwi_file)
        self.dwi_bval: File = File(self.dwi_bval)
        self.dwi_bvec: File = File(self.dwi_bvec)
        self.dwi_json: File = File(self.dwi_json)
        self.template: NiiFile = NiiFile(self.template)
        self.template_brain: NiiFile = NiiFile(self.template_brain)
        self.labels: NiiFile = NiiFile(self.labels)
        self.log: LogFile = LogFile(self.log)
            
    def mask_dwi(self,
                frac_int: float = 0.5
                ) -> Tuple[NiiFile,NiiFile,NiiFile]:
        '''doc-string'''
        [path, filename, ext] = self.dwi_file.file_parts()
        
        brain: str = os.path.join(path,filename + "_brain" + ext)
        mask: str = os.path.join(path,filename + "_brain_mask" + ext)
        head: str = os.path.join(path,filename + "_head" + ext)
        
        brain: NiiFile = NiiFile(brain)
        mask: NiiFile = NiiFile(mask)
        head: NiiFile = NiiFile(head)
        
        dwi = ReconMRtrix(self.dwi_file.file,
                         self.dwi_bval.file,
                         self.dwi_bvec.file,
                         self.dwi_json.file,
                         self.log)
        
        mif_file = dwi.nifti_to_mif()
        
        [mask_mif,brain_mif,head_mif] = dwi.create_mask(mif_file,frac_int)
        
        # Convert NIFTI to MIF
        mr_convert = Command("mrconvert")
        mr_convert.cmd_list.append(mask_mif.file)
        mr_convert.cmd_list.append(mask.file)
        mr_convert.run(self.log)
        
        mr_convert = Command("mrconvert")
        mr_convert.cmd_list.append(brain_mif.file)
        mr_convert.cmd_list.append(brain.file)
        mr_convert.run(self.log)
        
        mr_convert = Command("mrconvert")
        mr_convert.cmd_list.append(head_mif.file)
        mr_convert.cmd_list.append(head.file)
        mr_convert.run(self.log)
        
        # Clean-up
        os.remove(mask_mif.file)
        os.remove(brain_mif.file)
        os.remove(head_mif.file)
        
        return mask,brain,head
    
    def compute_linear_xfm(self,
                           dwi_brain: NiiFile,
                           dof: int = 12
                          ) -> Tuple[File,NiiFile]:
        '''doc-string'''
        [path, filename, ext] = self.dwi_file.file_parts()
        
        xfm_mat: str = os.path.join(path,filename + f".lin_xfm_{dof}_dof" + ".mat")
        xfm_out: str = os.path.join(path,filename + f".lin_xfm_{dof}_dof" + ext)
            
        xfm_mat: File = File(xfm_mat)
        xfm_out: NiiFile = NiiFile(xfm_out)
        
        if not dwi_brain.file:
            FSLError("DWI file not present.")
        
        lin_xfm = Command("flirt")
        lin_xfm.cmd_list.append("-in")
        lin_xfm.cmd_list.append(self.template_brain.abs_path())
        lin_xfm.cmd_list.append("-ref")
        lin_xfm.cmd_list.append(dwi_brain.abs_path())
        lin_xfm.cmd_list.append("-omat")
        lin_xfm.cmd_list.append(xfm_mat.file)
        lin_xfm.cmd_list.append("-out")
        lin_xfm.cmd_list.append(xfm_out.file)
        
        lin_xfm.run(self.log)
        
        return xfm_mat,xfm_out
    
    def compute_non_linear_xfm(self,
                               xfm_mat: File,
                               dwi_head: NiiFile
                              ) -> Tuple[NiiFile,NiiFile,NiiFile]:
        '''doc-string'''
        [path, filename, ext] = self.dwi_file.file_parts()
        
        nl_out: str = os.path.join(path,filename + f".non-lin_xfm" + ext)
        nl_warp: str = os.path.join(path,filename + f".non-lin_xfm.warp_field" + ext)
        nl_wpcf: str = os.path.join(path,filename + f".non-lin_xfm.warp_field_coeff" + ext)
        
        nl_out: NiiFile = NiiFile(nl_out)
        nl_warp: NiiFile = NiiFile(nl_warp)
        nl_wpcf: NiiFile = NiiFile(nl_wpcf)
        
        if not xfm_mat.file:
            FSLError("Linear transformation matrix not present.")
        
        if not dwi_head.file:
            FSLError("DWI file not present.")
        
        non_lin = Command("fnirt")
        non_lin.cmd_list.append(f"--in={self.template.abs_path()}")
        non_lin.cmd_list.append(f"--ref={dwi_head.abs_path()}")
        non_lin.cmd_list.append(f"--aff={xfm_mat.file}")
        non_lin.cmd_list.append(f"--iout={nl_out.file}")
        non_lin.cmd_list.append(f"--fout={nl_warp.file}")
        non_lin.cmd_list.append(f"--cout={nl_wpcf.file}")
        non_lin.run(self.log)
        
        return nl_out,nl_warp,nl_wpcf
    
    def applywarp(self,
                  dwi_head: NiiFile,
                  non_lin_warp: NiiFile,
                  interp: str = "",
                  rel: bool = True,
                  premat: Optional[File] = None,
                 ) -> NiiFile:
        '''doc-string'''
        [path, filename, ext] = self.dwi_file.file_parts()
        
        out: str = os.path.join(path,filename + f".labels.non-linear" + ext)
        out: NiiFile = NiiFile(out)
        
        if not dwi_head.file:
            FSLError("DWI file not present.")
        
        if not non_lin_warp.file:
            FSLError("Warp file not present.")
        
        warp = Command("applywarp")
        warp.cmd_list.append(f"--in={self.labels.abs_path()}")
        warp.cmd_list.append(f"--ref={dwi_head.abs_path()}")
        warp.cmd_list.append(f"--warp={non_lin_warp.abs_path()}")
        warp.cmd_list.append(f"--out={out.file}")
        
        if premat:
            warp.cmd_list.append(f"--premat={premat.abs_path()}")
        
        if interp:
            warp.cmd_list.append(f"--interp={interp}")
        
        if rel:
            warp.cmd_list.append("--rel")
        else:
            warp.cmd_list.append("--abs")
        
        warp.run(self.log)
        
        return out

In [6]:
class ReconDTK(object):
    '''class doc-string'''
    
    def __init__(self):
        '''doc-string'''
    
    def dti_fit(dwi,mask,bvec,bval,threshold):
        '''doc-string'''
    
    def create_mask(dwi,bval):
        '''doc-string'''
    
    def extract_b0s(dwi,bval):
        '''doc-string'''
        # DWIxfm.extract_b0s
    
    def dti_recon(dwi,mask,threshold):
        '''doc-string'''
    
    # write visualization functions later

In [5]:
f = "../sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.nii.gz"
b = "../sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.bval"
e = "../sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.bvec"
j = "../sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.json"
log = "sub-test.log"

In [6]:
template = "../UNC_AAL/infant-neo_1mm.nii"
labels = "../UNC_AAL/infant-neo-aal.nii"
template_brain = "../UNC_AAL/infant-neo_1mm_brain.nii"

In [7]:
work_tmp = TmpDir("work.tmp",use_cwd=True)

In [10]:
# work_tmp.rm_tmp_dir(rm_parent=True)
# os.remove(log)

In [8]:
f_tmp = work_tmp.TmpFile(os.path.basename(f),work_tmp.tmp_dir)
b_tmp = work_tmp.TmpFile(os.path.basename(b),work_tmp.tmp_dir)
e_tmp = work_tmp.TmpFile(os.path.basename(e),work_tmp.tmp_dir)
j_tmp = work_tmp.TmpFile(os.path.basename(j),work_tmp.tmp_dir)

In [9]:
work_tmp.mk_tmp_dir()

In [10]:
copy(f,f_tmp.file)
copy(b,b_tmp.file)
copy(e,e_tmp.file)
copy(j,j_tmp.file)

'/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.json'

In [11]:
dwi = DWIxfm(f_tmp.file,
             b_tmp.file,
             e_tmp.file,
             template,
             template_brain,
             labels,
             log,
             j_tmp.file)

In [12]:
mask,brain,head = dwi.mask_dwi()

Running: mrconvert -json_import /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.json -fslgrad /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.bvec /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.bval /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.nii.gz /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.mif
Running: mrconvert -json_import /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.json -fslgrad /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.bvec /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti

Running: mrconvert /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01__brain.mif /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_brain.nii.gz
Running: mrconvert /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01__brain.mif /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_brain.nii.gz
mrconvert: [100%] copying from "/mnt/c/Use...val-b2000_run-01__brain.mif" to "/mnt/c/Use...000_run-01_dwi_brain.nii.gz"
mrconvert: [100%] compressing image "/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_brain.nii.gz"

mrconvert: [100%] copying from "/mnt/c/Use...val-b2000_run-01__brain.mif" to "/mnt/c/Use...000_run-01_dwi_brain.nii.gz"
mrconvert: [100%] compressing 

In [13]:
dwi.template_brain.abs_path()

'/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/UNC_AAL/infant-neo_1mm_brain.nii'

In [14]:
brain.file

'/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_brain.nii.gz'

In [15]:
mask.file

'/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_brain_mask.nii.gz'

In [16]:
xfm_mat, xfm_out = dwi.compute_linear_xfm(brain,dof=12)

Running: flirt -in /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/UNC_AAL/infant-neo_1mm_brain.nii -ref /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_brain.nii.gz -omat /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.lin_xfm_12_dof.mat -out /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.lin_xfm_12_dof.nii.gz
Running: flirt -in /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/UNC_AAL/infant-neo_1mm_brain.nii -ref /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_brain.nii.gz -omat /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.lin_xfm_12_dof.mat -out /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_d

In [17]:
xfm_mat.file

'/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.lin_xfm_12_dof.mat'

In [18]:
nl_out,nl_warp,nl_wpcf = dwi.compute_non_linear_xfm(xfm_mat,head)

Running: fnirt --in=/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/UNC_AAL/infant-neo_1mm.nii --ref=/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_head.nii.gz --aff=/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.lin_xfm_12_dof.mat --iout=/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.non-lin_xfm.nii.gz --fout=/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.non-lin_xfm.warp_field.nii.gz --cout=/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.non-lin_xfm.warp_field_coeff.nii.gz
Running: fnirt --in=/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/UNC_AAL/infant-neo_1mm.nii --ref=/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data

In [19]:
labels = dwi.applywarp(head,nl_warp,"nn",True)

Running: applywarp --in=/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/UNC_AAL/infant-neo-aal.nii --ref=/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_head.nii.gz --warp=/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.non-lin_xfm.warp_field.nii.gz --out=/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.labels.non-linear.nii.gz --interp=nn --rel
Running: applywarp --in=/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/UNC_AAL/infant-neo-aal.nii --ref=/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_head.nii.gz --warp=/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.non-lin_xfm.warp_field.nii.gz --out=/mnt/c/Users/smart/Desktop/IR

In [20]:
labels.file

'/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.labels.non-linear.nii.gz'

In [ ]:
# MRtrix

In [75]:
mm = ReconMRtrix(f_tmp.file,
                 b_tmp.file,
                 e_tmp.file,
                 j_tmp.file,
                 log)

In [22]:
mm.nii_file.file

'/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.nii.gz'

In [23]:
mf = mm.Mif(mm.nii_file.file)
mf.file

'/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.mif'

In [24]:
mi = mm.nifti_to_mif(force=True)

Running: mrconvert -force -json_import /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.json -fslgrad /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.bvec /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.bval /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.nii.gz /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.mif
Running: mrconvert -force -json_import /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.json -fslgrad /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.bvec /mnt/c/Users/smart/Desktop/I

In [25]:
mi.file

'/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.mif'

In [26]:
[wm_res,gm_res,csf_res] = mm.estimate_response(mif=mi,force=True)

Running: dwi2response -force dhollander /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.mif /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_response_wm.txt /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_response_gm.txt /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_response_csf.txt
Running: dwi2response -force dhollander /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.mif /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_response_wm.txt /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_respons

In [27]:
up_mif = mm.mr_upsample(mif=mi,vox=1.5)

Running: mrgrid /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.mif regrid /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_upsampled.mif -voxel 1.5
Running: mrgrid /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.mif regrid /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_upsampled.mif -voxel 1.5
Running: mrgrid /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.mif regrid /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_upsampled.mif -voxel 1.5



 	 The required dependency mrresize is not installed or in the system path. 



IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [28]:
mask,brain,head = mm.create_mask(up_mif,cleanup=True)

Running: dwiextract -bzero /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_upsampled.mif /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/tmp_dir_7542/tmp_B0s.nii.gz
Running: dwiextract -bzero /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_upsampled.mif /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/tmp_dir_7542/tmp_B0s.nii.gz
Running: dwiextract -bzero /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_upsampled.mif /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/tmp_dir_7542/tmp_B0s.nii.gz
dwiextract: [100%] extracting volumes
dwiextract: [100%] compressing image "/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/tmp_dir_7542/tmp_B0s.nii.gz"


In [32]:
# os.remove(mask.file); os.remove(brain.file)

In [29]:
[wm_fod, gm, csf] = mm.ss3t_csd(mif=up_mif, 
                             mask=mask, 
                             wm_res=wm_res, 
                             gm_res=gm_res, 
                             csf_res=csf_res)

Running: ss3t_csd_beta1 /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_upsampled.mif /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_response_wm.txt /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_upsamp_wm_fod.mif /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_response_gm.txt /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_upsamp_gm_tis.mif /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_response_csf.txt /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_upsamp_csf_tis.mif -mask /mnt/c/Users

In [30]:
[wm_fod_norm, gm_norm, csf_norm] = mm.bias_field_correction(wm_fod=wm_fod, 
                                                            gm_tis=gm, 
                                                            csf_tis=csf, 
                                                            mask=mask)

Running: mtnormalise /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_upsamp_wm_fod.mif /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_wm_fod_norm.mif /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_upsamp_gm_tis.mif /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_gm_tis_norm.mif /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_upsamp_csf_tis.mif /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_csf_tis_norm.mif -mask /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_upsamp_brain_mask.mif
Running

In [31]:
[dec,vf] = mm.compute_dec_map(wm_fod_norm, gm_norm, csf_norm,mask)

Running: fod2dec /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_wm_fod_norm.mif /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_dec.mif -mask /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_upsamp_brain_mask.mif
Running: fod2dec /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_wm_fod_norm.mif /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_dec.mif -mask /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_upsamp_brain_mask.mif
Running: fod2dec /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_wm_fo

In [32]:
dec.file
vf.file

'/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_vf.mif'

In [33]:
tcks = mm.mr_tck_global(wm_fod_norm,mask)

Running: tckgen /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_wm_fod_norm.mif /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.100000.streamlines.tck -seed_image /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_upsamp_brain_mask.mif -select 100000 -cutoff 0.07
Running: tckgen /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_wm_fod_norm.mif /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.100000.streamlines.tck -seed_image /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_upsamp_brain_mask.mif -select 100000 -cutoff 0.07
Running: tckgen /mnt/c/Users/smart/Desktop/IRC

In [34]:
tcks.file

'/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.100000.streamlines.tck'

In [35]:
tcks_filt = mm.mr_tck_sift(tcks,wm_fod_norm,10000,mask)

Running: tcksift /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.100000.streamlines.tck /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_wm_fod_norm.mif /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.10000.streamlines.filtered.tck -proc_mask /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_upsamp_brain_mask.mif -term_number 10000
Running: tcksift /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.100000.streamlines.tck /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_wm_fod_norm.mif /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub

In [44]:
b_tmp.file

'/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_6014/sub-C01_ses-001_bval-b2000_run-01_dwi.bval'

In [ ]:
# Copy bvals and bvecs for QIT visualization
b_tmp = work_tmp.TmpFile(os.path.basename(b),work_tmp.tmp_dir)
e_tmp = work_tmp.TmpFile(os.path.basename(e),work_tmp.tmp_dir)

In [ ]:
copy(b_tmp.file,)
copy(e_tmp.file.)

In [76]:
connectome,fa_connectome,md_connectome,ad_connectome,rd_connectome = mm.structural_connectome(tcks,
                                                                                              labels,
                                                                                              up_mif,
                                                                                              mask,
                                                                                              fa = True,
                                                                                              md = True,
                                                                                              ad = True,
                                                                                              rd = True,
                                                                                              force = True)

Running: dwi2tensor /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_upsampled.mif /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.diff_tensor.mif -mask /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_upsamp_brain_mask.mif -force
Running: dwi2tensor /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_upsampled.mif /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.diff_tensor.mif -mask /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi_upsamp_brain_mask.mif -force
Running: dwi2tensor /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_

Running: tensor2metric /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.diff_tensor.mif -adc /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.md.mif -fa /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.fa.mif -ad /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.ad.mif -rd /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.rd.mif
Running: tensor2metric /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.diff_tensor.mif -adc /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.md.mif -fa /mnt/c/Users/smart/De

tck2connectome: [WARNING] existing output files will be overwritten
tck2connectome: Image "/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.labels.non-linear.nii.gz" stored with floating-point type; need to check for non-integer or negative values
tck2connectome: [100%] uncompressing image "/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.labels.non-linear.nii.gz"
tck2connectome: [100%] Verifying parcellation image
tck2connectome: [100%] uncompressing image "/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.labels.non-linear.nii.gz"
tck2connectome: [100%] Constructing connectome



tck2connectome: [WARNING] existing output files will be overwritten
tck2connectome: Image "/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.labels.non-linear.nii.gz" stored with floating-point type; need to check for non-integer or negative values
tck2connectome: [100%] uncompressing image "/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.labels.non-linear.nii.gz"
tck2connectome: [100%] Verifying parcellation image
tck2connectome: [100%] uncompressing image "/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.labels.non-linear.nii.gz"
tck2connectome: [100%] Constructing connectome



tck2connectome: [WARNING] existing output files will be overwritten
tck2connectome: Image "/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.labels.non-linear.nii.gz" stored with floating-point type; need to check for non-integer or negative values
tck2connectome: [100%] uncompressing image "/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.labels.non-linear.nii.gz"
tck2connectome: [100%] Verifying parcellation image
tck2connectome: [100%] uncompressing image "/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.labels.non-linear.nii.gz"
tck2connectome: [100%] Constructing connectome



...

tck2connectome: Image "/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.labels.non-linear.nii.gz" stored with floating-point type; need to check for non-integer or negative values
tck2connectome: [100%] uncompressing image "/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.labels.non-linear.nii.gz"
tck2connectome: [100%] Verifying parcellation image
tck2connectome: [100%] uncompressing image "/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.labels.non-linear.nii.gz"
tck2connectome: [100%] Constructing connectome



tck2connectome: Image "/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.labels.non-linear.nii.gz" stored with floating-point type; need to check for non-integer or negative values
tck2connectome: [100%] uncompressing image "/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.labels.non-linear.nii.gz"
tck2connectome: [100%] Verifying parcellation image
tck2connectome: [100%] uncompressing image "/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.labels.non-linear.nii.gz"
tck2connectome: [100%] Constructing connectome



tck2connectome: Image "/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.labels.non-linear.nii.gz" stored with floating-point type; need to check for non-integer or negative values
tck2connectome: [100%] uncompressing image "/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.labels.non-linear.nii.gz"
tck2connectome: [100%] Verifying parcellation image
tck2connectome: [100%] uncompressing image "/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.labels.non-linear.nii.gz"
tck2connectome: [100%] Constructing connectome



tck2connectome: Image "/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.labels.non-linear.nii.gz" stored with floating-point type; need to check for non-integer or negative values
tck2connectome: [100%] uncompressing image "/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.labels.non-linear.nii.gz"
tck2connectome: [100%] Verifying parcellation image
tck2connectome: [100%] uncompressing image "/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.labels.non-linear.nii.gz"
tck2connectome: [100%] Constructing connectome



tck2connectome: Image "/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.labels.non-linear.nii.gz" stored with floating-point type; need to check for non-integer or negative values
tck2connectome: [100%] uncompressing image "/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.labels.non-linear.nii.gz"
tck2connectome: [100%] Verifying parcellation image
tck2connectome: [100%] uncompressing image "/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397/sub-C01_ses-001_bval-b2000_run-01_dwi.labels.non-linear.nii.gz"
tck2connectome: [100%] Constructing connectome



In [55]:
# copy data in work_tmp to another directory when finished

In [47]:
work_tmp.rm_tmp_dir(rm_parent=True)

In [46]:
1.2e6

1200000.0

In [47]:
a = False
b = False
c = False

In [48]:
if a or b or c:
    print(True)

In [54]:
a: List[File]

In [56]:
a = [1,2,3]
b = [4,5,6]

In [58]:
for i in zip(a,b):
    print(i[0])
    print(i[1])

1
4
2
5
3
6


In [79]:
work_tmp.tmp_dir

'/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/work.tmp/tmp_dir_8397'